In [2]:
!pip install yfinance

In [22]:
#first get the data
import yfinance as yf  
import pandas as pd
from datetime import date, datetime
from pandas import DataFrame
from pandas import concat

apple = yf.download('AAPL','2016-01-01', date.today())

[*********************100%***********************]  1 of 1 downloaded


In [23]:
apple.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-12-31,107.01,107.03,104.82,105.26,98.66,40912300
2016-01-04,102.61,105.37,102.00,105.35,98.74,67649400
2016-01-05,105.75,105.85,102.41,102.71,96.27,55791000
2016-01-06,100.56,102.37,99.87,100.70,94.38,68457400
2016-01-07,98.68,100.13,96.43,96.45,90.40,81094400


In [24]:
apple.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 942 entries, 2015-12-31 to 2019-09-27
Data columns (total 6 columns):
Open         942 non-null float64
High         942 non-null float64
Low          942 non-null float64
Close        942 non-null float64
Adj Close    942 non-null float64
Volume       942 non-null int64
dtypes: float64(5), int64(1)
memory usage: 51.5 KB


In [25]:
apple.describe()

,Open,High,Low,Close,Adj Close,Volume
count,942.000000,942.000000,942.000000,942.000000,942.000000,9.420000e+02
mean,156.489639,157.871603,155.185860,156.575042,152.679926,3.233241e+07
std,38.948132,39.360274,38.529086,38.928389,40.180779,1.465967e+07
min,90.000000,91.670000,89.470000,90.340000,85.650000,1.136200e+07
25%,115.867500,116.735000,115.602500,116.185000,111.152500,2.249618e+07
50%,159.825000,160.985000,158.485000,159.855000,155.250000,2.825450e+07
75%,187.242500,188.870000,185.887500,187.465000,184.620000,3.750168e+07
max,230.780000,233.470000,229.780000,232.070000,228.520000,1.333697e+08


In [26]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [27]:
import numpy as np
apple['Close'].values
apple.shape[1]

6

In [28]:
new_apple = series_to_supervised(pd.DataFrame(apple['Close']),5, 1,False)

In [29]:
new_apple.head()

,var1(t-5),var1(t-4),var1(t-3),var1(t-2),var1(t-1),var1(t)
Date,,,,,,
2015-12-31,NaN,NaN,NaN,NaN,NaN,105.26
2016-01-04,NaN,NaN,NaN,NaN,105.26,105.35
2016-01-05,NaN,NaN,NaN,105.26,105.35,102.71
2016-01-06,NaN,NaN,105.26,105.35,102.71,100.70
2016-01-07,NaN,105.26,105.35,102.71,100.70,96.45


In [30]:
new_apple_full = pd.concat([apple, new_apple], axis=1)

In [31]:
#drop var1(t) since this is same as the close value
new_apple_full.drop(columns=['var1(t)'], inplace=True)

In [32]:
new_apple_full

,Open,High,Low,Close,Adj Close,Volume,var1(t-5),var1(t-4),var1(t-3),var1(t-2),var1(t-1)
Date,,,,,,,,,,,
2015-12-31,107.01,107.03,104.82,105.26,98.66,40912300,NaN,NaN,NaN,NaN,NaN
2016-01-04,102.61,105.37,102.00,105.35,98.74,67649400,NaN,NaN,NaN,NaN,105.26
2016-01-05,105.75,105.85,102.41,102.71,96.27,55791000,NaN,NaN,NaN,105.26,105.35
2016-01-06,100.56,102.37,99.87,100.70,94.38,68457400,NaN,NaN,105.26,105.35,102.71
2016-01-07,98.68,100.13,96.43,96.45,90.40,81094400,NaN,105.26,105.35,102.71,100.70
...,...,...,...,...,...,...,...,...,...,...,...
2019-09-23,218.95,219.84,217.65,218.72,218.72,19165500,219.90,220.70,222.77,220.96,217.73
2019-09-24,221.03,222.49,217.19,217.68,217.68,31190800,220.70,222.77,220.96,217.73,218.72
2019-09-25,218.55,221.50,217.14,221.03,221.03,21903400,222.77,220.96,217.73,218.72,217.68


In [48]:
#shift the close values to label up and down movements
new_apple_full['previous-close'] = pd.DataFrame(new_apple_full['Close']).shift(1).values

In [54]:
new_apple_full

,Open,High,Low,Close,Adj Close,Volume,var1(t-5),var1(t-4),var1(t-3),var1(t-2),var1(t-1),previous-close
Date,,,,,,,,,,,,
2015-12-31,107.01,107.03,104.82,105.26,98.66,40912300,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-04,102.61,105.37,102.00,105.35,98.74,67649400,NaN,NaN,NaN,NaN,105.26,105.26
2016-01-05,105.75,105.85,102.41,102.71,96.27,55791000,NaN,NaN,NaN,105.26,105.35,105.35
2016-01-06,100.56,102.37,99.87,100.70,94.38,68457400,NaN,NaN,105.26,105.35,102.71,102.71
2016-01-07,98.68,100.13,96.43,96.45,90.40,81094400,NaN,105.26,105.35,102.71,100.70,100.70
...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-23,218.95,219.84,217.65,218.72,218.72,19165500,219.90,220.70,222.77,220.96,217.73,217.73
2019-09-24,221.03,222.49,217.19,217.68,217.68,31190800,220.70,222.77,220.96,217.73,218.72,218.72
2019-09-25,218.55,221.50,217.14,221.03,221.03,21903400,222.77,220.96,217.73,218.72,217.68,217.68


In [57]:
new_apple_full['movement'] = np.where(new_apple_full['Close'] > new_apple_full['previous-close'], 'up', 'down')

In [58]:
new_apple_full

,Open,High,Low,Close,Adj Close,Volume,var1(t-5),var1(t-4),var1(t-3),var1(t-2),var1(t-1),previous-close,movement
Date,,,,,,,,,,,,,
2015-12-31,107.01,107.03,104.82,105.26,98.66,40912300,NaN,NaN,NaN,NaN,NaN,NaN,down
2016-01-04,102.61,105.37,102.00,105.35,98.74,67649400,NaN,NaN,NaN,NaN,105.26,105.26,up
2016-01-05,105.75,105.85,102.41,102.71,96.27,55791000,NaN,NaN,NaN,105.26,105.35,105.35,down
2016-01-06,100.56,102.37,99.87,100.70,94.38,68457400,NaN,NaN,105.26,105.35,102.71,102.71,down
2016-01-07,98.68,100.13,96.43,96.45,90.40,81094400,NaN,105.26,105.35,102.71,100.70,100.70,down
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-23,218.95,219.84,217.65,218.72,218.72,19165500,219.90,220.70,222.77,220.96,217.73,217.73,up
2019-09-24,221.03,222.49,217.19,217.68,217.68,31190800,220.70,222.77,220.96,217.73,218.72,218.72,down
2019-09-25,218.55,221.50,217.14,221.03,221.03,21903400,222.77,220.96,217.73,218.72,217.68,217.68,up


In [59]:
#save it as CSV
new_apple_full.to_csv('prepared-prices.csv')